# Попрактикуемся с тем, что изучили

Будем практиковаться на датасете:
https://www.kaggle.com/c/avito-demand-prediction

Ваша задача:
1. Создать Dataset для загрузки данных (используем только числовые данные)
2. Обернуть его в Dataloader
3. Написать архитектуру сети, которая предсказывает число показов на основании числовых данных (вы всегда можете нагенерить дополнительных факторов). Сеть должна включать BatchNorm слои и Dropout (или НЕ включать, но нужно обосновать)
4. Учить будем на функцию потерь с кагла (log RMSE) - нужно её реализовать
5. Сравните сходимость Adam, RMSProp и SGD, сделайте вывод по качеству работы модели

train-test разделение нужно сделать с помощью sklearn random_state=13, test_size = 0.25

Вопросы? в личку @Kinetikm

# Реализация

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import torch.nn.functional as F
import torch.nn as nn

import torch

from sklearn.metrics import mean_squared_log_error

## Class Dataset

In [35]:
class MyDataset(torch.utils.data.Dataset):
    """class MyDataset 
    convert all categorical fields to numbers (categorical codes)
    """
    def __init__(self, init_dataset):
#         self._base_dataset = init_dataset
        self._base_dataset = self.transform(init_dataset)
    
    def __len__(self):
#         return len(self._base_dataset)
        return self._base_dataset.shape[0]

    def __getitem__(self, idx):
        return np.array(self._base_dataset[self._base_dataset.columns.drop('deal_probability')].iloc[idx].values, dtype=np.double), self._base_dataset['deal_probability'].iloc[idx]
                
    @staticmethod    
    def transform(df):
        if set(['image', 'image_top_1']).issubset(train_dataset.columns):
            df.drop(['image', 'image_top_1'], axis=1, inplace=True)
        for name in df.select_dtypes(include=['object', 'datetime64[ns]']).columns:
            df[name] = df[name].astype('category')
            cat_name = name + '_code'
            df[cat_name] = df[name].cat.codes
            df.drop([name], axis=1, inplace=True)
        return df

## Load source date

In [3]:
train_dataset = pd.read_csv("HW_data/train.csv", parse_dates=["activation_date"])

## DataLoader

In [36]:
dataset = MyDataset(train_dataset)

train_loader = torch.utils.data.DataLoader(dataset,
                          batch_size=128,
                          shuffle=True,
                          num_workers=0)

for batch, label in train_loader:
    print(batch.shape)
    break

torch.Size([128, 15])


## Construct net

In [60]:
class Perceptron(nn.Module):
    def __init__(self, input_dim, output_dim, activation="relu"):
        super(Perceptron, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.activation = activation
        
    def forward(self, x):
        x = self.fc(x)
        if self.activation=="relu":
            return F.relu(x)
        if self.activation=="sigmoid":
            return F.sigmoid(x)
        raise RuntimeError
        

class FeedForward(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FeedForward, self).__init__()
        self.bn1 = nn.BatchNorm1d(input_dim)
        self.fc1 = Perceptron(input_dim, hidden_dim)
        self.bn2 = nn.BatchNorm1d(hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc2 = nn.Sigmoid()
        
    def forward(self, x):
        x = self.bn1(x)
        x = self.fc1(x)
        x = self.bn2(x)
        x = self.dp(x)
        x = self.fc2(x)
        return x

## Optimizer and loss function

In [61]:
net = FeedForward(15, 10)

optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# use mean_squared_log_error from sklearn.metrics 

## Education

___Не разобрался почему не пошло обучение. Прошу подсказать.___

In [62]:
for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = torch.Tensor(np.double(data[0])), data[1]
        print(f'inputs: {inputs}')
        print(f'labels: {labels}')
        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        print(f'outpust: {outputs}')
        loss = mean_squared_log_error(outputs, labels)
        
        print(f'labels: {labels.shape}')
        print(f'outpust: {outputs}')
        
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            running_loss = 0.0

print('Training is finished!')

  0%|                                                                                           | 0/10 [00:05<?, ?it/s]

inputs: tensor([[5.5000e+03, 3.4000e+01, 1.0956e+06,  ..., 1.0537e+06, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 2.0000e+00, 1.3327e+06,  ..., 1.0175e+06, 0.0000e+00,
         1.0000e+00],
        [3.5000e+03, 3.3000e+01, 5.8670e+05,  ..., 8.7773e+05, 1.2000e+01,
         1.0000e+00],
        ...,
        [8.0000e+02, 3.0000e+01, 1.4976e+06,  ..., 1.1402e+05, 9.0000e+00,
         1.0000e+00],
        [1.0000e+03, 2.9100e+02, 8.4320e+05,  ..., 1.1351e+06, 1.0000e+01,
         1.0000e+00],
        [3.8000e+03, 7.3000e+01, 7.2132e+05,  ..., 1.0761e+06, 2.0000e+00,
         1.0000e+00]])
labels: tensor([0.1387, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.5226, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.7376,
        0.0000, 0.8652, 0.0000, 0.0000, 0.1575, 0.0000, 0.0000, 0.0000, 0.2746,
        0.0000, 0.0000, 0.5000, 0.0000, 0.0799, 0.0000, 0.1992, 0.0000, 0.0000,
        0.0649, 0.2603, 0.0000, 0.3087, 0.0000, 0.0000, 0.0000, 0.00

RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.